
# 小地域の収入推定

```Data```フォルダには、小地域の収入データ(```income_district.csv```)と国勢調査の小地域集計データが格納されています。

国勢調査の小地域集計データ（人口構成、労働力構成、住宅形態など）から、その地域の収入を推定することが考えられます。推定のための機械学習・深層学習モデルを構築しなさい。


- データを観察・理解する上で、データの構造を説明しながら、適切なデータ整形を行いなさい
- データ構造や分析結果に対して、少なくとも二つの図で可視化を行いなさい
- モデルの精度を評価し、できるだけ精度が高いモデルを得るよう、適切な特徴量エンジニアリングやモデル選定の考えもまとめなさい


In [2]:
import geopandas as gpd
import pandas as pd

# GeoJSONファイルの読み込み
geo_data = gpd.read_file('13-tokyo-all.geojson')

# 収入データと国勢調査の小地域集計データ
directory = 'Data' # ディレクトリのパス

age_data_path = f'{directory}/h27_age_df.csv'
family_data_path = f'{directory}/h27_family_df.csv'
gender_data_path = f'{directory}/h27_gender_df2.csv'
housing_data_path = f'{directory}/h27_house_df.csv'
house_info_data_path = f'{directory}/h27_house_info_df.csv'
industry_data_path = f'{directory}/h27_indusry_df.csv'
occupation_data_path = f'{directory}/h27_job_df.csv'
labor_data_path = f'{directory}/h27_labor_df.csv'
marriage_data_path = f'{directory}/h27_marriage_df.csv'
employment_status_data_path = f'{directory}/h27_work_status_df.csv'
income_data_path = f'{directory}/income_district.csv'

age_data = pd.read_csv(age_data_path) #年齢層別人口、性別別人口などに関するデータ
family_data = pd.read_csv(family_data_path) #家族に関するデータ 
gender_data = pd.read_csv(gender_data_path) #総人口、男女別人口などに関するデータ
housing_data = pd.read_csv(housing_data_path) #住宅関連データ
house_info_data = pd.read_csv(house_info_data_path) #住居の階数別データ
industry_data = pd.read_csv(industry_data_path) #職業別男女人口データ
occupation_data = pd.read_csv(occupation_data_path) #職業別男女人口データ（職業分類が異なる）
labor_data = pd.read_csv(labor_data_path) #労働状況データ
marriage_data = pd.read_csv(marriage_data_path) #結婚状況データ
employment_status_data = pd.read_csv(employment_status_data_path) #就業形態データ
income_data = pd.read_csv(income_data_path) #地域コードと平均収入



In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# `gender_data` から `population` 列を抽出し、数値に変換
gender_data['population'] = pd.to_numeric(gender_data['population'], errors='coerce')
gender_data = gender_data.dropna(subset=['population'])

# `income_data` から `income_mean` 列を抽出し、数値に変換
income_data['income_mean'] = pd.to_numeric(income_data['income_mean'], errors='coerce')
income_data = income_data.dropna(subset=['income_mean'])

# データの確認
print(gender_data.head())
print(income_data.head())

# 説明変数と目的変数の設定（ここではデータのマージをせず、個別に処理）
# ここでは一例として `gender_data` の最初の1000行を使用して `income_data` と同様に設定
X = gender_data[['population']].head(1000)  # 説明変数
y = income_data['income_mean'].head(1000)   # 目的変数

# トレーニングデータとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 線形回帰モデルの作成
model = LinearRegression()
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# モデルの評価
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# 回帰係数
print("Coefficient:", model.coef_[0])
print("Intercept:", model.intercept_)


   district_id  district2_id  level_identifier state_name city_name  \
0        18201           NaN                 1        福井県       福井市   
1        18201          10.0                 2        福井県       福井市   
3        18201          40.0                 2        福井県       福井市   
4        18201        4001.0                 3        福井県       福井市   
5        18201        4002.0                 3        福井県       福井市   

  district_name district2_name  population male_population female_population  \
0           NaN            NaN    265904.0          128892            137012   
1           明里町            NaN       386.0             191               195   
3            足羽            NaN      2881.0            1382              1499   
4            足羽            １丁目       771.0             375               396   
5            足羽            ２丁目       653.0             301               352   

  family_count  
0        99872  
1          193  
3         1178  
4          314  
5      

/var/folders/tk/yz7rjm5j5yv8mt2w6k846r0c0000gp/T/ipykernel_39740/3232020250.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_data['population'] = pd.to_numeric(gender_data['population'], errors='coerce')
